In [1]:
# Import modules
import pandas as pd
import numpy as np
from datetime import datetime

# Display all dataframe columns
pd.set_option('display.max_columns', None)

## Source #1: Con Ed Public January 2020

In [2]:
# Read CSV
coned_df = pd.read_csv('data/csv/con_ed_public_january_2020.csv')
coned_df.head(3)

,Company,Developer,Application / Job #,Division,City/Town,Zip Code,Circuit ID,Substation,Hybrid (Y/N),Related Application/Job #,PV (kW),ESS (kW),Wind (kW),MT (kW),SG (kW),IG (kW),FW (kW),FC (kW),CHP (kW),GT (kW),HYDRO (kW),ICE (kW),ST (kW),Other (kW),Metering,Value Stack (Y/N),Application Date,Application End Date,Final Letter of Acceptance Date,Project Complete (Y/N),Unnamed: 30
0,CECONY,MOMENTUM SOLAR,MC-468889,CENY-Q,WOODSIDE,11377.0,SN NWK,Newtown,N,NaN,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,NaN,31-Jan-20,NaN,NaN,No,NaN
1,CECONY,1st Light Energy Inc.,MC-469001,CENY-Q,Ozone Park,11417.0,9854,Brownsville_2,N,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,NaN,31-Jan-20,NaN,NaN,No,NaN
2,CECONY,"Sunrun, Inc.",MC-469012,CENY-BX,Brooklyn,11236.0,3031,Bensonhurst_2,N,NaN,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,NaN,31-Jan-20,31-Jan-20,NaN,No,NaN


In [3]:
# Shape 
coned_df.shape

(35479, 31)

In [4]:
# Keep only ESS (not include thermal/ice storage)
coned_df = coned_df[pd.notnull(coned_df['ESS (kW)'])]
coned_df = pd.DataFrame(coned_df.loc[coned_df['ICE (kW)'].isnull()])

# Keep columns
coned_df = pd.DataFrame(coned_df[[
    'Developer', 'Application / Job #', 'Division', 'Zip Code', 'Substation', 
    'ESS (kW)', 'Application Date', 'Final Letter of Acceptance Date', 'Project Complete (Y/N)'
]])

# Rename columns
coned_df = coned_df.rename(columns = {
    'Developer': 'developer',
    'Application / Job #': 'coned_id',
    'Division': 'borough',
    'Zip Code': 'zipcode',
    'Substation': 'substation',
    'ESS (kW)': 'power_kw',
    'Application Date': 'application_date', 
    'Final Letter of Acceptance Date': 'final_loa_date',
    'Project Complete (Y/N)': 'project_complete'
})

# Drop if 'Division' = 'CENY-W'
coned_df = coned_df[coned_df.borough !='CENY-W']

# Rename 'Borough' names
coned_df.loc[(coned_df.borough == 'CENY-BK'), 'borough'] = 'Brooklyn'
coned_df.loc[(coned_df.borough == 'CENY-BX'), 'borough'] = 'Bronx'
coned_df.loc[(coned_df.borough == 'CENY-M'), 'borough'] = 'Manhattan'
coned_df.loc[(coned_df.borough == 'CENY-Q'), 'borough'] = 'Queens'
coned_df.loc[(coned_df.borough == 'CENY-SI'), 'borough'] = 'Staten Island'

# Convert values to int
coned_df['zipcode'] = coned_df['zipcode'].astype(int)
coned_df['power_kw'] = pd.to_numeric(coned_df['power_kw'].astype(int))

# Convert to datetime
coned_df['application_date'] = pd.to_datetime(coned_df['application_date'])
coned_df['final_loa_date'] = pd.to_datetime(coned_df['final_loa_date'])

# Completed projects df
coned_completed_df = coned_df[coned_df.project_complete == 'Yes']

In [5]:
# All projects
coned_df.to_csv('data/clean/csv/coned_all_df.csv', index=False)
coned_df.head(3)

,developer,coned_id,borough,zipcode,substation,power_kw,application_date,final_loa_date,project_complete
74,Raiden Electric,MC-468401,Brooklyn,10019,Newtown,5000,2020-01-29,NaT,No
91,SunPower Corp.,MC-468591,Queens,11430,Brownsville_2,2500,2020-01-29,NaT,No
92,SunPower Corp.,MC-468599,Queens,11430,Brownsville_2,2500,2020-01-29,NaT,No


In [6]:
# Completed projects
coned_completed_df.to_csv('data/clean/csv/coned_completed_df.csv', index=False)
coned_completed_df.head(3)

,developer,coned_id,borough,zipcode,substation,power_kw,application_date,final_loa_date,project_complete
11201,"EnerNOC, Inc.",MC-372959,Brooklyn,10001,Brownsville_2,2475,2018-09-14,2019-07-01,Yes
13428,EnerNOC,MC-352787,Brooklyn,10001,Brownsville_2,2475,2018-06-04,2019-07-01,Yes
17782,Green Power Solutions,MC-297318,Bronx,10471,Sherman_Creek,133,2017-08-18,2018-11-29,Yes


In [7]:
#### Count total projects
print(f'Total Projects:', coned_df['coned_id'].count())
print()

# Count completed projects
print(f'Completed Projects:')
print(coned_df['project_complete'].value_counts())
print()

# Count total projects per borough
print('All Projects by Borough:')
print(coned_df['borough'].value_counts())
print()

# Developers
#print(f'Developers:')
#print(coned_df['Developer'].value_counts())
#print()

# Substations
#print('Total Applications per Substation:')
#print(coned_df['Substation'].value_counts())

Total Projects: 79

Completed Projects:
No     57
Yes    22
Name: project_complete, dtype: int64

All Projects by Borough:
Brooklyn         32
Manhattan        18
Queens           13
Staten Island    10
Bronx             6
Name: borough, dtype: int64



## Source #2: NYISO Queue

In [8]:
# Read CSV
nyiso_df = pd.read_csv('data/csv/nyiso_interconnection_queue.csv')
nyiso_df.head(3)

,Queue Pos.,Owner/Developer,Project Name,Date of IR,SP (MW),WP (MW),Type/ Fuel,County,State,Z,Interconnection Point,Utility,S,Last Update,Availability of Studies,FS Complete/ SGIA Tender,Proposed In-Service,Proposed Initial-Sync,Proposed COD
0,5,Sithe Energies,Torne Valley Station,1/28/99,860,NaN,NaN,NaN,NaN,NaN,Ramapo,CONED,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,6,Sunset Energy Fleet LLC,Sunset Energy Fleet,2/17/99,520,NaN,NaN,NaN,NaN,NaN,Gowanus,CONED,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7,American National Power,Ramapo Energy,2/23/99,1100,NaN,NaN,NaN,NaN,NaN,Ramapo,CONED,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Shape 
nyiso_df.shape

(891, 19)

In [10]:
# Keep columns
nyiso_df = pd.DataFrame(nyiso_df[[
    'Queue Pos.', 'Owner/Developer', 'Project Name', 'Date of IR', 'SP (MW)', 'Type/ Fuel', 'County', 'Z',
    'Interconnection Point',
]])

# Rename columns
nyiso_df = nyiso_df.rename(columns = {
    'Queue Pos.': 'nyiso_id',
    'Owner/Developer': 'developer',
    'Project Name': 'project_name',
    'Date of IR': 'ir_date',
    'SP (MW)': 'power_kw',
    'Type/ Fuel': 'type',
    'County': 'borough',
    'Z': 'zone',
    'Interconnection Point': 'substation',
})

# Keep only energy storage projects in Zone J
nyiso_df = nyiso_df[nyiso_df.type == 'ES']
nyiso_df = nyiso_df[nyiso_df.zone == 'J']

# Rename boroughs
nyiso_df.loc[(nyiso_df.borough == 'Kings'), 'borough'] = 'Brooklyn'
nyiso_df.loc[(nyiso_df.borough == 'New York'), 'borough'] = 'Manhattan'
nyiso_df.loc[(nyiso_df.borough == 'Richmond'), 'borough'] = 'Staten Island'

# Drop if Borough = Union
nyiso_df = nyiso_df[nyiso_df.borough !='Union']

# Convert to datetime
nyiso_df['ir_date'] = pd.to_datetime(nyiso_df['ir_date'])

# Write CSV
nyiso_df.to_csv('data/clean/csv/nyiso_df.csv', index=False)
nyiso_df.head(3)

,nyiso_id,developer,project_name,ir_date,power_kw,type,borough,zone,substation
440,522,NYC Energy LLC,NYC Energy,2015-12-16,79.9,ES,Manhattan,J,Hudson Avenue East 138kV
592,674,"Helix Ravenswood, LLC",Vernon Battery Storage I,2017-10-24,10,ES,Queens,J,Vernon Substation
593,675,"Helix Ravenswood, LLC",Vernon Battery Storage II,2017-10-24,10,ES,Queens,J,Vernon Substation


In [11]:
# Count total projects
print(f'Total Projects:', nyiso_df['nyiso_id'].count())
print()

# Count total projects per borough
print('Projects by Borough:')
print(nyiso_df['borough'].value_counts())
print()

# Substations
#print('Total Applications per Substation:')
#print(coned_df['substation'].value_counts())

Total Projects: 38

Projects by Borough:
Queens           20
Manhattan        10
Brooklyn          4
Staten Island     4
Name: borough, dtype: int64



## Source #3a: NYSERDA - Projects

In [12]:
# Read CSV
nyserda_projects_df = pd.read_csv('data/csv/nyserda_der_metric_data_projects.csv')
nyserda_projects_df.head(3)

,Project ID,System ID,Facility ID,Facility Name,System Name,Project Name,NYSERDA Contract Name,Contracted Capacity (kW),Commercial Operation Date,Facility NYISO Zone,Technology,Project Rated Electric Generation (kW),Project Rated Heat Generation (MBtu/h),Project Rated Electric Discharge Capacity (kW),Project Rated Electric Storage Capacity (kWh),Project Rated Cooling Energy Discharge Capacity (ton),Project Equivalent Electrical Discharge Capacity from Cooling Sources (kW),Project Rated Cooling Energy Storage Capacity (ton-hour),Project Equivalent Electrical Storage Capacity from Cooling Sources (kWh),Project Rated Thermal Energy Discharge Capacity (MBtu/h),Project Equivalent Electrical Discharge Capacity from Thermal Sources (kW),Project Rated Thermal Energy Storage Capacity (MBtu),Project Equivalent Electrical Storage Capacity from Thermal Sources (kWh),Developer,Installed Resources,Reporting Start Date,Reporting End Date,Alternate Name(s),Value Streams,ConEd Target Zone,NYSERDA-funded,Heat Recovery to DHW,Heat Recovery to Space Heat,Heat Recovery to Chiller,Heat Recovery to Other Load,System Redundancy (N+1),Black-Start/Islanding Capable,Grid Export Capable
0,1,1,170,Waldbaum's Supermarket,NaN,Waldbaum's Supermarket,Waldbaum's Supermarket,NaN,NaN,K - Long Island,CHP,60.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,CDH Energy Corp.,1,2002-08-02,2006-07-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2,2,181,Arrow Linen,NaN,Arrow Linen,Arrow Linen,NaN,NaN,J - New York City,CHP,300.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,Energy Concepts,2,2005-03-01,2009-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,3,3,209,Modern Landfill,NaN,Modern Landfill,Modern Landfill,NaN,NaN,A - West,CHP,5600.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,Aria Energy (formerly Innovative Energy Systems),7,2004-12-31,2013-10-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automatic,True


In [13]:
# Shape
nyserda_projects_df.shape

(1213, 38)

In [14]:
# Keep columns
nyserda_projects_df = pd.DataFrame(nyserda_projects_df[[
    'Project ID', 'System ID', 'Facility ID', 'Facility Name', 'Project Name', 
    'Commercial Operation Date', 'Facility NYISO Zone', 'Technology', 
    'Project Rated Electric Discharge Capacity (kW)', 'Project Rated Electric Storage Capacity (kWh)', 'Project Rated Cooling Energy Storage Capacity (ton-hour)',
    'Developer', 'Value Streams', 'NYSERDA-funded',
]])

# Rename columns
nyserda_projects_df = nyserda_projects_df.rename(columns = {
    'Project ID': 'nyserda_project_id', 
    'System ID': 'nyserda_system_id', 
    'Facility ID': 'nyserda_facility_id',
    'Facility Name': 'facility_name', 
    'Project Name': 'project_name',
    'Commercial Operation Date': 'commercial_operation_date',
    'Facility NYISO Zone': 'zone',
    'Technology': 'technology',
    'Project Rated Electric Discharge Capacity (kW)': 'power_kw',
    'Project Rated Electric Storage Capacity (kWh)': 'energy_kwh',
    'Project Rated Cooling Energy Storage Capacity (ton-hour)': 'cooling_tonh',
    'Developer': 'developer',
    'Value Streams': 'value_streams',
    'NYSERDA-funded': 'nyserda_funded',
})

# Change to int, replace 0s with NaN
nyserda_projects_df['power_kw'] = pd.to_numeric(nyserda_projects_df['power_kw'].astype(int))
nyserda_projects_df['energy_kwh'] = pd.to_numeric(nyserda_projects_df['energy_kwh'].astype(int))
nyserda_projects_df = nyserda_projects_df.replace(0, np.nan)

# Keep only energy storage projects in Zone J, remove thermal storage
nyserda_projects_df = nyserda_projects_df[nyserda_projects_df.technology == 'Energy Storage']
nyserda_projects_df = nyserda_projects_df[nyserda_projects_df.zone == 'J - New York City']
nyserda_projects_df = nyserda_projects_df[pd.notnull(nyserda_projects_df['power_kw'])]
nyserda_projects_df = nyserda_projects_df[pd.notnull(nyserda_projects_df['energy_kwh'])]

# Convert to datetime
nyserda_projects_df['commercial_operation_date'] = pd.to_datetime(nyserda_projects_df['commercial_operation_date'])

# Write CSV
nyserda_projects_df.to_csv('data/clean/csv/nyserda_projects_df.csv', index=False)
nyserda_projects_df.head(3)

,nyserda_project_id,nyserda_system_id,nyserda_facility_id,facility_name,project_name,commercial_operation_date,zone,technology,power_kw,energy_kwh,cooling_tonh,developer,value_streams,nyserda_funded
520,567,620,635,Glenwood - Liberty Plaza,NaN,2016-08-01,J - New York City,Energy Storage,100.0,400.0,NaN,"Enel X (fka Demand Energy, EnerNOC)",Con Ed Demand Mgt Program; Demand Charge Reduc...,True
521,568,621,636,Glenwood - Emerald Green,NaN,2016-08-01,J - New York City,Energy Storage,100.0,400.0,NaN,"Enel X (fka Demand Energy, EnerNOC)",Con Ed Demand Mgt Program; Demand Charge Reduc...,True
522,569,622,637,Glenwood - Barclay Tower,NaN,2013-06-01,J - New York City,Energy Storage,225.0,2000.0,NaN,"Enel X (fka Demand Energy, EnerNOC)",Con Ed Demand Mgt Program; Demand Charge Reduc...,NaN


In [15]:
# Count total projects
print(f'Total Projects:', nyserda_projects_df['nyserda_project_id'].count())
print()

Total Projects: 22



## Source #3b: NYSERDA - Facilities

In [16]:
# Read CSV
nyserda_facilities_df = pd.read_csv('data/csv/nyserda_der_metric_data_facilities.csv')
nyserda_facilities_df.head(3)

,Facility ID,Facility Name,Address Line 1,Address Line 2,City,State,ZIP,Latitude,Longitude,Total Rated Electric Generation (kW),Total Rated Heat Generation (MBtu/h),Total Rated Electrical Discharge Capacity (kW),Total Rated Electrical Storage Capacity (kWh),Total Rated Cooling Energy Discharge Capacity (ton),Total Equivalent Electrical Discharge Capacity from Cooling Sources (kW),Total Rated Cooling Energy Storage Capacity (ton-hour),Total Equivalent Electrical Storage Capacity from Cooling Sources (kWh),Total Rated Thermal Energy Discharge Capacity (MBtu/h),Total Equivalent Electrical Discharge Capacity from Thermal Sources (kW),Total Rated Thermal Energy Storage Capacity (MBtu),Total Equivalent Electrical Storage Capacity from Thermal Sources (kWh),Facility Submits Data on Electricity Imported from Utility?,Source Weather Station for Ambient Temperature Data,Metric Data Last Modified,Installed Systems,Alternate Name(s),Facility Website (external),Category,Subcategory,Floor Area (ft²),No. of Occupancy Units,Electric Utility,Gas Utility,NYISO Zone,Located in Flood Plain
0,5,Aurora Ridge Dairy,2542 Angling Rd,NaN,Aurora,NY,13026,43.031,-76.467,600.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,False,KITH,2/21/19 16:05,1,NaN,NaN,Agricultural,Dairy,87120000.0,NaN,New York State Electric and Gas,New York State Electric and Gas,C - Central,NaN
1,11,Emerling Farm,2616 NY-246,NaN,Perry,NY,14530,42.758,-78.006,200.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,False,KROC,2/21/19 16:03,1,NaN,NaN,Agricultural,Dairy,NaN,NaN,New York State Electric and Gas,New York State Electric and Gas,A - West,NaN
2,23,River Point Towers,555 Kappock Ave,NaN,Bronx,NY,10463,40.882,-73.917,300.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,True,KLGA,2/21/19 16:04,1,NaN,NaN,Residential,Multifamily,NaN,NaN,Consolidated Edison,Consolidated Edison,J - New York City,NaN


In [17]:
# Shape
nyserda_facilities_df.shape

(1174, 35)

In [18]:
# Keep columns
nyserda_facilities_df = pd.DataFrame(nyserda_facilities_df[[
    'Facility ID', 'Facility Name', 'Address Line 1', 'City', 'State', 'ZIP', 'Latitude', 'Longitude',
    'Total Rated Electrical Discharge Capacity (kW)', 'Total Rated Electrical Storage Capacity (kWh)',
    'Category', 'Subcategory', 'NYISO Zone',
]])

# Rename columns
nyserda_facilities_df = nyserda_facilities_df.rename(columns = { 
    'Facility ID': 'nyserda_facility_id',
    'Facility Name': 'facility_name', 
    'Address Line 1': 'address',
    'City': 'city',
    'State': 'state',
    'ZIP': 'zipcode',
    'latitude': 'latitude',
    'longitude': 'longitude',
    'NYISO Zone': 'zone',
    'Total Rated Electrical Discharge Capacity (kW)': 'power_kw',
    'Total Rated Electrical Storage Capacity (kWh)': 'energy_kwh',
    'Category': 'category',
    'Subcategory': 'subcategory',
})

# Replace 0's with NaN
nyserda_facilities_df = nyserda_facilities_df.replace(0, np.nan)

# Keep only energy storage projects in Zone J, remove thermal storage
nyserda_facilities_df = nyserda_facilities_df[pd.notnull(nyserda_facilities_df['energy_kwh'])]
nyserda_facilities_df = nyserda_facilities_df[nyserda_facilities_df.zone == 'J - New York City']

# Concatenate coordinates for Leaflet
nyserda_facilities_df['coordinates'] = '[' + nyserda_facilities_df['Latitude'].map(str) + ', ' + nyserda_facilities_df['Longitude'].map(str) + ']'

# Write CSV
nyserda_facilities_df.to_csv('data/clean/csv/nyserda_facilities_df.csv', index=False)
nyserda_facilities_df.head(3)

,nyserda_facility_id,facility_name,address,city,state,zipcode,Latitude,Longitude,power_kw,energy_kwh,category,subcategory,zone,coordinates
351,422,Marcus Garvey Apartments,353 Chester St,Brooklyn,NY,11212,40.663061,-73.909792,300.0,1200.0,Residential,Multifamily,J - New York City,"[40.663061, -73.90979200000001]"
527,635,Glenwood - Liberty Plaza,10 Liberty St,New York,NY,10005,40.707395,-74.010326,100.0,400.0,Residential,Multifamily,J - New York City,"[40.707395, -74.01032632]"
528,636,Glenwood - Emerald Green,320 W 38th St,New York,NY,10018,40.755092,-73.995227,100.0,400.0,Residential,Multifamily,J - New York City,"[40.7550922, -73.9952269]"


In [19]:
# Count total projects
print(f'Total Projects:', nyserda_facilities_df['nyserda_facility_id'].count())
print()

# Count total projects per borough
print('Projects by Borough:')
print(nyiso_df['borough'].value_counts())
print()

Total Projects: 20

Projects by Borough:
Queens           20
Manhattan        10
Brooklyn          4
Staten Island     4
Name: borough, dtype: int64

